In [3]:
# Basic Ranked Retrieval (RRI)
import pickle
import numpy as np
import pandas as pd
from xml.dom import minidom
from xml.etree import cElementTree as ElementTree
import os
import nltk
import ssl
import math
import csv

try:
     _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
     pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/isabelasarmiento/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/isabelasarmiento/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/isabelasarmiento/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def indexReader():
    """
    Reads the inverted index created in the distributed_inverted_index.ipynb notebook
    The path from where it reads the file is docs/inverted_index.pkl
    :return: the inverted index, represented in a dictionary where the keys are the terms
    and the values is also a dictionary that contains the frecuency of documents
    that contain the term, and the posting. The posting is an array itself
    that contains the document id, and the term's frequency in that document.
    {'term': {'freq': df, 'posting':[[doc1, tf1],[doc2, tf2],...,[docn, tfn]}}
    """
    with open('docs/inverted_index.pkl', 'rb') as index:
        return pickle.load(index)

invertedIndex = indexReader()


In [5]:
## Imports
def documentReader():
    """
    This method reads the documents
    :return: Dictionary of documents {dXXX: content of document dXXX}
    """
    documents_path = os.path.join(os.getcwd(), 'docs/docs-raw-texts')
    documentos = {}
    documents_paths = os.listdir(documents_path)
    documents_paths.sort()
    #print(documents_paths)
    for filename in documents_paths:
        file_path = os.path.join(documents_path, filename)
        #print(filename)
        xmldoc = minidom.parse(file_path)
        id = xmldoc.getElementsByTagName('public')[0].attributes['publicId'].value
        title = xmldoc.getElementsByTagName('fileDesc')[0].attributes['title'].value
        data = next(ElementTree.parse(file_path).iter('raw')).text
        documentos[id] = (title + ' ' + data).replace(u'\xa0', u' ').replace('\n', ' ')
    return documentos

documentos = documentReader()
print(list(documentos.items())[0])

('d001', 'William Beaumont and the Human Digestion William Beaumont and the Human Digestion.  William Beaumont: Physiology of digestion Image Source.  On November 21, 1785, US-American surgeon William Beaumont was born. He became best known as “Father of Gastric Physiology” following his research on human digestion. William Beaumont was born in Lebanon, Connecticut and became a physician. He served as a surgeon’s mate in the Army during the War of 1812. He opened a private practice in Plattsburgh, New York, but rejoined the Army as a surgeon in 1819. Beaumont was stationed at Fort Mackinac on Mackinac Island in Michigan in the early 1820s when it existed to protect the interests of the American Fur Company. The fort became the refuge for a wounded 19-year-old French-Canadian fur trader named Alexis St. Martin when a shotgun went off by accident in the American Fur Company store at close range June 6th, 1822. St. Martin’s wound was quite serious because his stomach was perforated and se

In [6]:
def queries_reader():
    """
    This method reads the queries
    :return: Dictionary of queries {qYY: content of query qYY}
    """
    queries_path = os.path.join(os.getcwd(), 'docs/queries-raw-texts')
    queries = {}
    queries_paths = os.listdir(queries_path)
    queries_paths.sort()
    #print(documents_paths)
    queries_index = []
    for filename in queries_paths:
        file_path = os.path.join(queries_path, filename)
        #print(filename)
        xmldoc = minidom.parse(file_path)
        id = xmldoc.getElementsByTagName('public')[0].attributes['publicId'].value
        query = next(ElementTree.parse(file_path).iter('raw')).text
        queries[id] = query.replace(u'\xa0', u' ').replace('\n', ' ')
        queries_index.append(id)
    return queries, queries_index

queries, queries_index = queries_reader()
print(list(queries.items())[0])

('q01', 'Fabrication of music instruments')


In [7]:
def queries_tokenization(queries):
    """
    Queries tokenization using nltk. The returned dictionary,
    :param queries in a dictionary, {qYY: content of query qYY}
    :return: Dictionary of tokenized queries {qYY: ["term1","term2",...,"termn"]}
    """
    nltk_stop_words_en = set(nltk.corpus.stopwords.words("english"))
    wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()
    #print("items", queries.items())
    tokenized_queries = {}
    for key,doc in queries.items():
        word_tok = nltk.word_tokenize(doc)
        word_tok_sw = [token for token in word_tok if (token.isalnum()) and (token not in nltk_stop_words_en)
                       ]
        nltk_lemmaList = [wordnet_lemmatizer.lemmatize(word) for word in word_tok_sw]
        #print(nltk_lemmaList)
        tokenized_queries[key] = nltk_lemmaList

    return tokenized_queries



tokenized_queries = queries_tokenization(queries)
print(list(tokenized_queries.items())[0])

('q01', ['Fabrication', 'music', 'instrument'])


In [8]:
def basic_ranked_retrieval(queries,invertedIndex,documents,N):
    """
    Computes the score for all the queries
    :param queries: tokenized queries in a dictorionary {qYY: ["term1","term2",...,"termn"]}
    :param invertedIndex: inverted index in a dictionary {'term': {'freq': df, 'posting':[[doc1, tf1],[doc2, tf2],...,[docn, tfn]}}
    :param documents: tokenized documents in a dictionary {dXXX: content of document dXXX}
    :param N: total number of documents
    :return: a dictionary with the scores for all the queries for each document
    {'qYY': {'dXXX': score1, 'dXXX': score2, ..., 'dXXX': score3 } }
    """
    scores= {}
    query_scores_template = {}
    for key, doc in documents.items():
        query_scores_template[key] = 0

    for query,tokens in queries.items():
        query_scores = query_scores_template.copy()
        for token in tokens:
            if token in invertedIndex:
                df = invertedIndex[token]["freq"]
                idf = math.log10( N / df )
                for docs in invertedIndex[token]["posting"]:
                    tf = docs[1]
                    tf_w = math.log10(1 + tf)
                    #if docs[0] not in query_scores:
                    #    query_scores[docs[0]] = 0
                    docId = "d{0:0=3d}".format(docs[0])
                    query_scores[docId] += tf_w*idf
        clean_query_scores = { k : v for k,v in query_scores.items() if v != 0}
        clean_query_scores = dict(sorted(clean_query_scores.items(), key=lambda item: item[1], reverse=True))
        scores[query] = clean_query_scores#query_scores

    return scores

RRI = basic_ranked_retrieval(tokenized_queries,invertedIndex, documentos, len(documentos))
print(list(RRI.items())[0])

('q01', {'d254': 1.3322084124448144, 'd016': 1.280133266247014, 'd085': 0.7608038472948082, 'd185': 0.7210322829593981, 'd209': 0.7210322829593981, 'd060': 0.6881765238016224, 'd100': 0.6881765238016224, 'd153': 0.6881765238016224, 'd186': 0.6553207646438466, 'd006': 0.571404565150006, 'd215': 0.571404565150006, 'd099': 0.5193294189522057, 'd243': 0.5193294189522057, 'd004': 0.36051614147969907, 'd039': 0.36051614147969907, 'd065': 0.36051614147969907, 'd094': 0.36051614147969907, 'd130': 0.36051614147969907, 'd136': 0.36051614147969907, 'd152': 0.36051614147969907, 'd162': 0.36051614147969907, 'd164': 0.36051614147969907, 'd184': 0.36051614147969907, 'd195': 0.36051614147969907, 'd312': 0.36051614147969907, 'd316': 0.36051614147969907, 'd028': 0.3276603823219233, 'd038': 0.3276603823219233, 'd074': 0.3276603823219233, 'd082': 0.3276603823219233, 'd116': 0.3276603823219233, 'd170': 0.3276603823219233, 'd172': 0.3276603823219233, 'd212': 0.3276603823219233, 'd229': 0.3276603823219233, '

In [9]:
def writeScoreFile(RRI):
    """
    Writes the RRI-queries_results.tsv that contains the score of each query
    :param RRI: Dictionary with the scores, {'qYY': {'dXXX': score1, 'dXXX': score2, ..., 'dXXX': score3 } }
    :return: none
    """
    file_path = os.path.join(os.getcwd(), 'docs/answer_files/RRI-queries_results.tsv')
    with open(file_path, 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        for query_id,scores in RRI.items():
            scores_list = ""
            for doc,score in scores.items():
                scores_list+= doc +":"+str(round(score,4))+","
            tsv_writer.writerow([query_id,scores_list[:-1]])


writeScoreFile(RRI)
print("Archivo escrito")

Archivo escrito


In [34]:
def score(query, document_id, invertedIndex, N):
    """
    Computes to score for a query, document pair
    :param query: tokenized query in a document,  {qYY: ["term1","term2",...,"termn"]}
    :param document_id: id of the document in "dxxx" notation
    :param invertedIndex: inverted index in a dictionary, {'term': {'freq': df, 'posting':[[doc1, tf1],[doc2, tf2],...,[docn, tfn]}}
    :param N: the amount of documents
    :return: the score of the query (decimal)
    """
    score = 0

    tokens = query[1]
    for token in tokens:
        token_in_index = invertedIndex.get(token, "unknown_token")
        if token_in_index != "unknown_token":
            token_posting = token_in_index["posting"]
            tf = 0
            tf_w = 0
            for doc_freq_pair in token_posting:
                if doc_freq_pair[0] == document_id:
                    tf = doc_freq_pair[1]
                    tf_w = math.log10(1 + tf)
                    break
            df = token_in_index["freq"]
            idf = math.log10( N / df )
            score += tf_w*idf
    return  score


query1 = list(tokenized_queries.items())[3]

scoreQ1 = score(query1, 'd001', invertedIndex, len(documentos))
print(scoreQ1)

0.0


In [35]:
def read_judgemnts_file():
    """
    Reads the judgments file
    :return: the query scores in a dictionary
    {'qXX': {'dYYY': score }}
    """
    document_path = os.path.join(os.getcwd(), 'docs/relevance-judgments.tsv')
    tsv_file = open(document_path)
    read_tsv = csv.reader(tsv_file, delimiter="\t")
    relevance = {}
    for row in read_tsv:
        documents = row[1].split(',')
        query_relevance = {pair.split(':')[0] : int(pair.split(':')[1]) for pair in documents }
        query_relevance = dict(sorted(query_relevance.items(), key=lambda item: item[0]))
        relevance[row[0]] = query_relevance
    return relevance

relevance = read_judgemnts_file()
print(relevance)


{'q01': {'d016': 5, 'd186': 4, 'd254': 5}, 'q02': {'d136': 2, 'd139': 2, 'd143': 4, 'd147': 2, 'd149': 2, 'd164': 4, 'd228': 4, 'd283': 4, 'd291': 4, 'd293': 4, 'd318': 2}, 'q03': {'d105': 2, 'd147': 3, 'd152': 3, 'd283': 4, 'd291': 4, 'd318': 2}, 'q04': {'d010': 3, 'd019': 2, 'd049': 2, 'd270': 3, 'd275': 3, 'd286': 2, 'd330': 2}, 'q06': {'d026': 4, 'd069': 2, 'd233': 3, 'd257': 2, 'd297': 3, 'd329': 5}, 'q07': {'d004': 3, 'd077': 3, 'd179': 3, 'd266': 2}, 'q08': {'d005': 4, 'd028': 3, 'd081': 2, 'd108': 3, 'd110': 4, 'd117': 3, 'd121': 2, 'd180': 2, 'd205': 2, 'd251': 5, 'd271': 3, 'd292': 2}, 'q09': {'d177': 2, 'd198': 3, 'd199': 5, 'd205': 3, 'd217': 2, 'd223': 2}, 'q10': {'d052': 2, 'd065': 3, 'd068': 2, 'd076': 3, 'd100': 2, 'd199': 4, 'd215': 2, 'd231': 4}, 'q12': {'d239': 4, 'd250': 4, 'd258': 3, 'd277': 4}, 'q13': {'d049': 4, 'd056': 4, 'd239': 2, 'd258': 2, 'd277': 2}, 'q14': {'d002': 2, 'd005': 3, 'd041': 3, 'd081': 4, 'd091': 4, 'd093': 3, 'd117': 2, 'd130': 3, 'd142': 2, '

In [36]:
def precision_at_k(relevance: list, k: int):
    """
    DocString
    :return: Nothing
    """
    if k == 0:
        return 0
    l = np.array(relevance[:k]).sum()/k
    return l

def recall_at_k(relevance: list, nr_relevant: int, k: int):
    """
    :param relevance:
    :param nr_relevant:
    :param k:
    :return:
    """
    l = np.array(relevance[:k]).sum()/nr_relevant
    return l

def average_precision(relevance,R):

    length = len(relevance)
    sum = 0
    for i in range(length):
        if relevance[i]:
            sum += precision_at_k(relevance, i+1)
    return sum / R if R!=0 else 0

def mean_avg_precision(l):
    """
    DocString
    :return: Nothing
    """
    average = 0
    i = 1
    for lista,R in l:
        print(i)
        i+=1
        print("lista", lista)
        print("R:", R)
        print("Encontró todos", R==np.array(lista).sum())
        print("precisionav", average_precision(lista,R))
        average+= average_precision(lista,R)
    print(average)
    mean = average / len(l)
    return mean

#l = [1,0,1,1,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,1]
#print(type(l))
#prueba = average_precision(l)
#print(prueba)

def dcg_at_k(relevance, k: int):
    """
    DocString
    :return: Nothing
    """
    sum = 0
    i =  0
    for rel_i in relevance[: k]:
        i+= 1
        sum += rel_i/np.log2(max(i, 2))

    return sum

def ndcg_at_k(relevance, rel_sorted, k):
    """
    DocString
    :return: Nothing
    """
    #rel_sorted = sorted(relevance, reverse=True)
    max = dcg_at_k(rel_sorted, k)
    real = dcg_at_k(relevance, k)

    return real/ max if max != 0 else 0

In [37]:
def make_binary_score(query_tuple,relevance):

    query = query_tuple[0]
    ranking = query_tuple[1]
    binary_score = []
    M = len(relevance[query])
    i=1;
    for document, score in ranking.items():
        if i>M:
            break
        if document in relevance[query]:
            binary_score.append(1)
        else:
            binary_score.append(0)
        i += 1
    #print(binary_score)
    return binary_score, M

In [38]:
def make_binary_score_extended(query_tuple,relevance):
    query = query_tuple[0]
    ranking = query_tuple[1]
    binary_score = []
    M = len(relevance[query])
    i=0;
    for document, score in ranking.items():
        if i==M:
            break
        if document in relevance[query]:
            binary_score.append(1)
            i += 1
        else:
            binary_score.append(0)
    #print(binary_score)
    return binary_score, M


In [39]:
def precision_for_RRI(RRI,relevance):
    """
    DocString
    :return: Nothing
    """
    precisions = {}
    for query in RRI.items():
        binary_score, M = make_binary_score(query,relevance)
        precisions[query[0]] = precision_at_k(binary_score,M)
    return precisions

precisions = precision_for_RRI(RRI,relevance)
print(precisions)

{'q01': 0.6666666666666666, 'q02': 0.5454545454545454, 'q03': 1.0, 'q04': 0.8571428571428571, 'q06': 0.8333333333333334, 'q07': 0.25, 'q08': 0.75, 'q09': 0.8333333333333334, 'q10': 0.5, 'q12': 0.75, 'q13': 0.6, 'q14': 0.4166666666666667, 'q16': 0.5, 'q17': 0.75, 'q18': 0.8571428571428571, 'q19': 0.5, 'q22': 0.5714285714285714, 'q23': 0.25, 'q24': 0.0, 'q25': 0.75, 'q26': 1.0, 'q27': 0.5, 'q28': 0.6666666666666666, 'q29': 0.4166666666666667, 'q32': 1.0, 'q34': 1.0, 'q36': 0.4, 'q37': 0.3333333333333333, 'q38': 0.375, 'q40': 0.7777777777777778, 'q41': 0.8571428571428571, 'q42': 0.6666666666666666, 'q44': 0.7, 'q45': 0.75, 'q46': 0.5}


In [40]:
def recall_for_RRI(RRI,relevance):
    """
    DocString
    :return: Nothing
    """
    recalls = {}
    for query in RRI.items():
        binary_score, M = make_binary_score(query,relevance)
        recalls[query[0]] = recall_at_k(binary_score,M,M)
    return recalls

recalls = recall_for_RRI(RRI,relevance)
print(recalls)



{'q01': 0.6666666666666666, 'q02': 0.5454545454545454, 'q03': 1.0, 'q04': 0.8571428571428571, 'q06': 0.8333333333333334, 'q07': 0.25, 'q08': 0.75, 'q09': 0.8333333333333334, 'q10': 0.5, 'q12': 0.75, 'q13': 0.6, 'q14': 0.4166666666666667, 'q16': 0.5, 'q17': 0.75, 'q18': 0.8571428571428571, 'q19': 0.5, 'q22': 0.5714285714285714, 'q23': 0.25, 'q24': 0.0, 'q25': 0.75, 'q26': 1.0, 'q27': 0.5, 'q28': 0.6666666666666666, 'q29': 0.4166666666666667, 'q32': 1.0, 'q34': 1.0, 'q36': 0.4, 'q37': 0.3333333333333333, 'q38': 0.375, 'q40': 0.7777777777777778, 'q41': 0.8571428571428571, 'q42': 0.6666666666666666, 'q44': 0.7, 'q45': 0.75, 'q46': 0.5}


In [41]:
def map_for_RRI(RRI,relevance):
    """
    DocString
    :return: Nothing
    """
    precisions = []
    for query in RRI.items():
        binary_score, M = make_binary_score_extended(query,relevance)
        precisions.append((binary_score,M))
    print(precisions)
    map = mean_avg_precision(precisions)
    return map

map = map_for_RRI(RRI,relevance)
print(map)

[([1, 1, 0, 0, 0, 0, 0, 0, 1], 3), ([1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 11), ([1, 1, 1, 1, 1, 1], 6), ([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1], 7), ([1, 1, 1, 1, 1, 0, 1], 6), ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 4), ([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 12), ([1, 1, 1, 1, 0, 1, 1], 6), ([1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [42]:
def make_non_binary_score(query_tuple,relevance):
    query = query_tuple[0]
    ranking = query_tuple[1]
    non_binary_score = []
    M = len(relevance[query])
    i=1;
    for document, score in ranking.items():
        if i>M:
            break
        if document in relevance[query]:
            non_binary_score.append(relevance[query][document])
        else:
            non_binary_score.append(0)
        i += 1
    rel_sorted = [rel for doc,rel in relevance[query].items() ]
    rel_sorted = sorted(rel_sorted, reverse=True)
    return non_binary_score, rel_sorted, M

query1 = list(RRI.items())[0]
prueba = make_non_binary_score(query1,relevance)
print(prueba)

([5, 5, 0], [5, 5, 4], 3)


In [43]:
def ndcg_for_RRI(RRI,relevance):
    """
    DocString
    :return: Nothing
    """
    ndcgs = {}
    for query in RRI.items():
        non_binary_score, rel_sorted, M  = make_non_binary_score(query,relevance)
        #print(non_binary_score)
        ndcgs[query[0]] = ndcg_at_k(non_binary_score, rel_sorted, M)
    return ndcgs

ndcg = ndcg_for_RRI(RRI,relevance)
print(ndcg)

av_ndgc = 0
for key,val in ndcg.items():
    av_ndgc += val
print(av_ndgc/len(ndcg))

{'q01': 0.7984848580994974, 'q02': 0.6372452055109703, 'q03': 0.9125990044855482, 'q04': 0.8375872815409342, 'q06': 0.9135933534799239, 'q07': 0.3373519727104165, 'q08': 0.8723596292045686, 'q09': 0.8233822119089756, 'q10': 0.474468013427548, 'q12': 0.7920776427801035, 'q13': 0.8326604750636971, 'q14': 0.45808996246537376, 'q16': 0.6, 'q17': 0.8770340301127121, 'q18': 0.923510185194647, 'q19': 0.7142857142857143, 'q22': 0.5988077017638601, 'q23': 0.6037460821441866, 'q24': 0.0, 'q25': 0.6626480272895834, 'q26': 1.0, 'q27': 0.7659193871914459, 'q28': 0.8086698088039842, 'q29': 0.6466281557910287, 'q32': 0.9947696772861268, 'q34': 1.0, 'q36': 0.5403756309698909, 'q37': 0.539848396117854, 'q38': 0.3732229932956358, 'q40': 0.7624284400989726, 'q41': 0.7698204997888964, 'q42': 0.8472668887613066, 'q44': 0.797139952349211, 'q45': 0.8922521020745832, 'q46': 0.7337495705923308}
0.7183435101311293


In [44]:
def RRI_simplifier(RRI):
    result = {}
    for qid,scores in RRI.items():
        docs = []
        for k,v in scores.items():
            docs.append(k)
        result[qid]=docs
    return result


results = RRI_simplifier(RRI)
results['q01'][:5]


['d254', 'd016', 'd085', 'd185', 'd209']

## Evaluation

In [45]:
def read_judgemnts_file():
    """
    Lee el archivo de relevancia de los jueces
    :return: Diccionario con pares key: value, donde el key es el id de cada query y el value
    es otro doccionario con las ids de los docs relevantes para esa query ordenados en forma creciente.
    """
    document_path = os.path.join(os.getcwd(), 'docs/relevance-judgments.tsv')
    tsv_file = open(document_path)
    read_tsv = csv.reader(tsv_file, delimiter="\t")
    relevance = {}
    for row in read_tsv:
        documents = row[1].split(',')
        query_relevance = {pair.split(':')[0] : pair.split(':')[1] for pair in documents }
        query_relevance = dict(sorted(query_relevance.items(), key=lambda item: item[0]))
        relevance[row[0]] = query_relevance
    return relevance


relevance = read_judgemnts_file()
print(relevance['q01'])

{'d016': '5', 'd186': '4', 'd254': '5'}


In [46]:
def make_binary_result(results, relevant_res):
    """
    Este método toma los resultados crudos obtenidos para las queries (Para cada query la lista de documentos ordenaos
    por relevancia), devuelve 3 representaciones de estos resultados. La primera es la representacion binaria at K.
    Que es del mismo tamaño que el número de documentos relevantes. La segunda es esta misma lista pero con la escala
    dada por el archivo de evaluación. La tercera está destinada al cálculo del MAP, tiene la representación binaria
    hasta que salgan todos los documentos relevantes o simplemente de todos los documentos, además en su segundo
    componente tiene el número de documentos relevantes que deberían salir en los resultados según el archivo de
    evaluación.
    :param results: Diccionario con resultados crudos de cada query. Ej: {'q01': ['d254', 'd016', 'd153', ...]}
    :param relevant_res: Las 3 representaciones antes mencionadas
    :return:
    """
    bin_relevant = {}
    rel_scale_repr = {}
    map_relevant_docs = {}
    for query, relevant_docs in relevant_res.items():
        bin_repr = []
        scaled_repr = []
        map_repr = []
        M = len(relevant_docs)
        for doc_id, rel_scale in relevant_docs.items():
            bin = 1 if doc_id in results[query][:M] else 0
            bin_repr.append(bin)
            scaled_repr.append(bin * int(rel_scale))
        i = 0
        for doc_id in results[query]:
            if i < M:
                map_bin = 1 if doc_id in relevant_res[query] else 0
                i += map_bin
                map_repr.append(map_bin)
        bin_relevant[query] = bin_repr
        rel_scale_repr[query] = scaled_repr
        map_relevant_docs[query] = [map_repr, M]
    return bin_relevant, rel_scale_repr, map_relevant_docs

bin_results, scaled_results, map_relevant_docs = make_binary_result(results, relevance)
print(bin_results['q01'])
print(scaled_results['q01'])

[1, 0, 1]
[5, 0, 5]


In [47]:
print('Primeros 5 documentos devueltos como relevantes para q01: \n', results['q01'][:5])
print('Documentos relevantes para q01 según jueces: \n' , relevance['q01'])
print('Representación binaria de q01, hasta el último doc relevante: \n' ,map_relevant_docs['q01'])

Primeros 5 documentos devueltos como relevantes para q01: 
 ['d254', 'd016', 'd085', 'd185', 'd209']
Documentos relevantes para q01 según jueces: 
 {'d016': '5', 'd186': '4', 'd254': '5'}
Representación binaria de q01, hasta el último doc relevante: 
 [[1, 1, 0, 0, 0, 0, 0, 0, 1], 3]


### Definition of IR metrics functions

In [48]:
def precision_at_k(relevance: list, k: int):
    """
    DocString
    :return: Nothing
    """
    if k == 0:
        return 0
    l = np.array(relevance[:k]).sum()/k
    return l

def recall_at_k(relevance: list, nr_relevant: int, k: int):
    """
    DocString
    :return: Nothing
    """
    l = np.array(relevance[:k]).sum()/nr_relevant
    return l

def average_precision(relevance):
    """
    DocString
    :return: Nothing
    """

    length = len(relevance[0])
    sum = 0
    for i in range(length):
        if relevance[0][i]:
            sum += precision_at_k(relevance[0], i+1)
    if np.array(relevance[0]).sum()==0:
        return 0
    else:
        return sum / relevance[1]

def mean_avg_precision(l):
    """
    DocString
    :return: Nothing
    """
    mean = np.array([ average_precision(lista) for lista in l]).mean()
    return mean

mean_avg_precision([[[0, 0, 0, 0, 0, 0, 1], 1], [[0, 0, 0, 1, 1], 2], [[0, 1, 0, 1, 1, 1, 1], 5]])

0.35468253968253965

In [49]:
def dcg_at_k(relevance, k: int):
    """
    Calcula el DCG at K de un vector binario representando los resultados relevantes para una query.
    :param relevance: Vector binario
    :return: DCG at K de nuestra query
    """

    sum = 0
    i =  0
    for rel_i in relevance[: k]:
        i+= 1
        sum += rel_i/np.log2(max(i, 2))

    return sum

dcg_at_k([4, 4, 3, 0, 0, 1, 3, 3, 3, 0], 6)

def ndcg_at_k(relevance, k):
    """
    Calcula el ndcg at k de un vector binario
    :return: NDCG at K.
    """
    rel_sorted = sorted(relevance, reverse=True)
    max = dcg_at_k(rel_sorted, k)
    real = dcg_at_k(relevance, k)

    return real/ max if max != 0 else 0


ndcg_at_k([4, 4, 3, 0, 0, 1, 3, 3, 3, 0], 6)

0.7424602308163405

In [50]:
print(recall_at_k(bin_results['q01'], 3, 3))


0.6666666666666666


## Compute Evaluation Metrics for each query

In [51]:
def evaluation_metric(bin_queries, query_index, scaled_results):
    """

    :param bin_queries: Diccionario con valores {query Key: vector}, donde el vector corresponde a una lista
    con la representación binaria de un de los resultados encontrados para una query con relación a los dados
    en el archivo de evaluación. Ej, para q01, los relevantes son: d186,d254,d016. El RRDV devuelve d254, d016,
    d153. Por ende, la representación binaria de q01, en el orden del archivo de evaluación es: [0, 1, 1]
    :param query_index: Lista con los ids de las queries. ['qo1', 'qo2', ...]
    :param scaled_results: Representación escalada de los resultados de las queries usando la escala dada en el
    archivo de evaluación. Ej, q01 pasa de [0, 1, 1] a [0, 5, 5]
    :return: Un dataframe con el cálculo del P@M, r@M y NDCG@M para cada query
    """
    COLUMNS = ['P@M', 'R@M', 'DCG@M', 'NDCG@M']
    records = []
    for query, bin_vec in bin_queries.items():
        scaled = scaled_results[query]
        M = len(bin_vec)
        pm = precision_at_k(bin_vec, M)
        rm = recall_at_k(bin_vec, M, M)
        dcg = dcg_at_k(scaled, M)
        ndcg = ndcg_at_k(scaled, M)
        records.append([pm, rm,dcg, ndcg])

    return pd.DataFrame.from_records(records, index=query_index, columns=COLUMNS)

metrics = evaluation_metric(bin_results, queries_index, scaled_results)
metrics

,P@M,R@M,DCG@M,NDCG@M
q01,0.666667,0.666667,8.154649,0.815465
q02,0.545455,0.545455,8.639302,0.656543
q03,1.000000,1.000000,11.389201,0.874220
q04,0.857143,0.857143,9.827595,0.933486
q06,0.833333,0.833333,10.119083,0.763466
q07,0.250000,0.250000,3.000000,1.000000
q08,0.750000,0.750000,15.025909,0.842140
q09,0.833333,0.833333,10.516002,0.894663
q10,0.500000,0.500000,5.307607,0.642423
q12,0.750000,0.750000,7.892789,0.797833


### MAP

In [52]:
def overall_map(map_relevant_docs):
    """
    Función que calcula el MAP de los resultados de las queries.
    :param map_relevant_docs: Vector binario de las queries asegurandose de que aparezcan todos los documentos relevantes
    :return: El Mean average precision de los resultados de las queries.
    """
    matrix = [vector for key, vector in map_relevant_docs.items() ]
    return mean_avg_precision(matrix)

print(f'MAP resultante de todas las queries: {overall_map(map_relevant_docs)}')


MAP resultante de todas las queries: 0.6706142431795926
